# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [142]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [143]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [144]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [145]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [146]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price, market_cap)

124.8 2137031157992


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [147]:
columns = ['Ticker', 'Stock Price', 'Market Cap', 'Shares to Buy']
df = pd.DataFrame(columns=columns)
df

,Ticker,Stock Price,Market Cap,Shares to Buy


In [148]:
df.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Cap,Shares to Buy
0,AAPL,124.8,2137031157992,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [149]:
df = pd.DataFrame(columns=columns)
base_url = 'https://sandbox.iexapis.com/stable'
tkn_param = f'token={IEX_CLOUD_API_TOKEN}'
for stock in stocks['Ticker'][:10]:
    api_url = f'{base_url}/stock/{stock}/quote?{tkn_param}'
    data = requests.get(api_url).json()
    price = data['latestPrice']
    market_cap = data['marketCap']
    #print(f'got {stock} {price}')
    df = df.append(
        pd.Series(
            [
                stock,
                price,
                market_cap,
                'N/A'
            ],
            index=columns
        ),
        ignore_index=True,
    )
    
df

,Ticker,Stock Price,Market Cap,Shares to Buy
0,A,125.41,38262110546,N/A
1,AAL,21.33,13794536130,N/A
2,AAP,161.46,10894697220,N/A
3,AAPL,125.86,2082687676874,N/A
4,ABBV,111.26,193856194352,N/A
5,ABC,102.95,20826285727,N/A
6,ABMD,333.98,15038342449,N/A
7,ABT,125.71,220091371659,N/A
8,ACN,260.80,170467484585,N/A
9,ADBE,460.64,220954654548,N/A


In [150]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [151]:
ticker_gorups = list(chunks(stocks['Ticker'], 100))
ticker_strings = []
for i in range(0, len(ticker_gorups)):
    ticker_strings.append(','.join(ticker_gorups[i]))
#print(ticker_strings)
df = pd.DataFrame(columns = columns)

for tkrs in ticker_strings:
    batch_api_call_url = f'{base_url}/stock/market/batch?symbols={tkrs}&types=quote&{tkn_param}'
    data = requests.get(batch_api_call_url).json()
    for tkr in tkrs.split(','):
        df = df.append(
            pd.Series(
                [
                    tkr,
                    data[tkr]['quote']['latestPrice'],
                    data[tkr]['quote']['marketCap'],
                    'N/A'
                    
                ],
                index=columns
            ),
            ignore_index=True,
        )
df
    

,Ticker,Stock Price,Market Cap,Shares to Buy
0,A,124.78,38916333833,N/A
1,AAL,20.98,13793607025,N/A
2,AAP,166.91,10746285569,N/A
3,AAPL,121.49,2129299127363,N/A
4,ABBV,109.32,199748407212,N/A
...,...,...,...,...
500,YUM,104.35,32521689394,N/A
501,ZBH,169.07,34993502187,N/A
502,ZBRA,505.59,26729307545,N/A
503,ZION,54.98,8961716083,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [152]:
portfolio_size = input("Enter the value of your portfolio: ")
try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Please enter the Number   ')

Enter the value of your portfolio: 100000
100000.0


In [153]:
position_size = val / len(df.index)
print(position_size)
for i in range(0, len(df.index)):
    stock_price = df.loc[i, 'Stock Price']
    df.loc[i, 'Shares to Buy'] = math.floor(position_size / stock_price)
df

198.01980198019803


,Ticker,Stock Price,Market Cap,Shares to Buy
0,A,124.78,38916333833,1
1,AAL,20.98,13793607025,9
2,AAP,166.91,10746285569,1
3,AAPL,121.49,2129299127363,1
4,ABBV,109.32,199748407212,1
...,...,...,...,...
500,YUM,104.35,32521689394,1
501,ZBH,169.07,34993502187,1
502,ZBRA,505.59,26729307545,0
503,ZION,54.98,8961716083,3


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [159]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
sheet_name='Balanced Portfolio'
df.to_excel(writer, sheet_name=sheet_name, index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [160]:
bg_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [161]:
columns

['Ticker', 'Stock Price', 'Market Cap', 'Shares to Buy']


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [162]:
column_formats = {
    'A': [columns[0], string_format],
    'B': [columns[1], dollar_format],
    'C': [columns[2], integer_format],
    'D': [columns[3], integer_format],
}

for col in column_formats.keys():
    writer.sheets[sheet_name].write(f'{col}1', column_formats[col][0], column_formats[col][1])
    writer.sheets[sheet_name].set_column(f'{col}:{col}', 18, column_formats[col][1])
    
    

## Saving Our Excel Output

Saving our Excel file is very easy:

In [163]:
writer.save()